In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 14:15:45.173084


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we can use the following parameters, decision variables, objective function, and constraints:

Parameters:
- Total required units: 239,600,480
- Setup and unit costs for each vendor (A-E)
- Maximum supply capacity for each vendor (A-E)

Decision Variables:
- x_A: Number of units purchased from vendor A
- x_B: Number of units purchased from vendor B
- x_C: Number of units purchased from vendor C
- x_D: Number of units purchased from vendor D
- x_E: Number of units purchased from vendor E

Objective Function:
The objective is to minimize the total cost, which includes both setup and unit costs. In mathematical terms, this can be expressed as:

minimize z = setup_cost_A + unit_cost_A * x_A/1000 + setup_cost_B + unit_cost_B * x_B/1000 + setup_cost_C + unit_cost_C * x_C/1000 + setup_cost_D + unit_cost_D * x_D/1000 + setup_cost_E + unit_cost_E * x_E/1000

Constraints:
1. Purchase quantities should not exceed the maximum supply capacity of each vendor:
   - 0 ≤ x_A ≤ 33,000,000
   - 0 ≤ x_B ≤ 160,000,000 (with special conditions for vendor B as per the problem statement)
   - 0 ≤ x_C ≤ 165,600,000
   - 0 ≤ x_D ≤ 12,000,000
   - 0 ≤ x_E ≤ 77,000,000 (with special conditions for vendor E as per the problem statement)

2. The total number of units purchased should equal the required number:
   - x_A + x_B + x_C + x_D + x_E = 239,600,480

3. Special conditions for vendor B:
   - If 22,000,000 ≤ x_B ≤ 70,000,000, then setup_cost_B = $125,804.84 and unit_cost_B = $68.099 per thousand units
   - If 70,000,001 ≤ x_B ≤ 100,000,000, then setup_cost_B = $269,304.84 and unit_cost_B = $66.049 per thousand units
   - If 100,000,001 ≤ x_B ≤ 150,000,000, then setup_cost_B = $464,304.84 and unit_cost_B = $64.099 per thousand units
   - If 150,000,001 ≤ x_B ≤ 160,000,000, then setup_cost_B = $761,304.84 and unit_cost_B = $62.119 per thousand units

4. Special conditions for vendor E:
   - If 0 ≤ x_E ≤ 42,000,000, then setup_cost_E = $0 and unit_cost_E = $70.150 per thousand units
   - If 42,000,001 ≤ x_E ≤ 77,000,000, then setup_cost_E = $84,000 and unit_cost_E decreases by 0.05% for every 1,000,000 units purchased (starting at $68.150 per thousand units)

This optimization model can be solved using linear or mixed-integer programming techniques to find the optimal allocation of units among the five vendors that minimizes the total cost.

In [7]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we can use the following parameters, decision variables, objective function, and constraints:

Parameters:
- Total required units: 239,600,480
- Setup and unit costs for each vendor (A-E)
- Maximum supply capacity for each vendor (A-E)

Decision Variables:
- x_A: Number of units purchased from vendor A
- x_B: Number of units purchased from vendor B
- x_C: Number of units purchased from vendor C
- x_D: Number of units purchased from vendor D
- x_E: Number of units purchased from vendor E

Objective Function:
The objective is to minimize the total cost, which includes both setup and unit costs. In mathematical terms, this can be expressed as:

minimize z = setup_cost_A + unit_cost_A * x_A/1000 + setup_cost_B + unit_cost_B * x_B/1000 + setup_cost_C + unit_cost_C * x_C/1000 + setup_cost_D + unit_cost_D * x_D/1000 + setup_cost_E + unit_cost_E * x_E/1000

Constraints:
1. Purchase quantities should not exceed the maximum supply 

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

To solve this problem using Pyomo, we'll first need to install the Pyomo package if you haven't done so already. You can install it via pip:

```
pip install pyomo
```

Now, let's create a Python script to define and solve the optimization problem using Pyomo. We'll use the sample data provided in the problem statement.

```python
import pyomo.environ as pyo

# Sample data
total_units = 239600480
max_supply = {
    'A': 33000000,
    'B': 160000000,
    'C': 165600000,
    'D': 12000000,
    'E': 77000000,
}
setup_cost = {
    'A': 0,
    'B': 0,  # Will be updated based on conditions
    'C': 0,
    'D': 0,
    'E': 0,  # Will be updated based on conditions
}
unit_cost = {
    'A': 68.15,
    'B': 0,  # Will be updated based on conditions
    'C': 68.15,
    'D': 68.15,
    'E': 70.15,  # Will be updated based on conditions
}

# Initialize the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, 6), domain=pyo.NonNegativeIntegers)

# Objective function
def obj_rule(model):
    return (
        setup_cost['A'] + unit_cost['A'] * model.x[1] / 1000 +
        setup_cost['B'] + unit_cost['B'] * model.x[2] / 1000 +
        setup_cost['C'] + unit_cost['C'] * model.x[3] / 1000 +
        setup_cost['D'] + unit_cost['D'] * model.x[4] / 1000 +
        setup_cost['E'] + unit_cost['E'] * model.x[5] / 1000
    )
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def supply_constraint_rule(model, i):
    return model.x[i] <= max_supply[list(max_supply.keys())[i - 1]]
model.supply_constraints = pyo.Constraint(range(1, 6), rule=supply_constraint_rule)

def total_units_constraint_rule(model):
    return sum(model.x[i] for i in range(1, 6)) == total_units
model.total_units_constraint = pyo.Constraint(rule=total_units_constraint_rule)

# Special conditions for vendor B
def vendor_B_conditions_rule(model):
    if 22000000 <= model.x[2] <= 70000000:
        setup_cost['B'] = 125804.84
        unit_cost['B'] = 68.099
    elif 70000001 <= model.x[2] <= 100000000:
        setup_cost['B'] = 269304.84
        unit_cost['B'] = 66.049
    elif 100000001 <= model.x[2] <= 150000000:
        setup_cost['B'] = 464304.84
        unit_cost['B'] = 64.099
    elif 150000001 <= model.x[2] <= 160000000:
        setup_cost['B'] = 761304.84
        unit_cost['B'] = 62.119
model.vendor_B_conditions = pyo.Constraint(rule=vendor_B_conditions_rule)

# Special conditions for vendor E
def vendor_E_conditions_rule(model):
    if 0 <= model.x[5] <= 42000000:
        setup_cost['E'] = 0
        unit_cost['E'] = 70.150
    elif 42000001 <= model.x[5] <= 77000000:
        setup_cost['E'] = 84000
        unit_cost['E'] = 68.150 - 0.0005 * ((model.x[5] - 42000001) // 1000000)
model.vendor_E_conditions = pyo.Constraint(rule=vendor_E_conditions_rule)

# Solve the optimization problem
solver = pyo.SolverFactory('glpk')
solution = solver.solve(model)

# Print the optimal solution
if solution.solver.status == pyo.SolverStatus.ok:
    print("Optimal solution found")
    print("Total cost:", pyo.value(model.obj))
    print("Units from each vendor:")
    for i in range(1, 6):
        print(f"Vendor {list(max_supply.keys())[i - 1]}: {pyo.value(model.x[i])}")
else:
    print("No optimal solution found")
```

This script defines the optimization problem using Pyomo and solves it using the GLPK solver. You can replace the solver with any other compatible solver supported by Pyomo.

In [11]:
print(response_2_text)

To solve this problem using Pyomo, we'll first need to install the Pyomo package if you haven't done so already. You can install it via pip:

```
pip install pyomo
```

Now, let's create a Python script to define and solve the optimization problem using Pyomo. We'll use the sample data provided in the problem statement.

```python
import pyomo.environ as pyo

# Sample data
total_units = 239600480
max_supply = {
    'A': 33000000,
    'B': 160000000,
    'C': 165600000,
    'D': 12000000,
    'E': 77000000,
}
setup_cost = {
    'A': 0,
    'B': 0,  # Will be updated based on conditions
    'C': 0,
    'D': 0,
    'E': 0,  # Will be updated based on conditions
}
unit_cost = {
    'A': 68.15,
    'B': 0,  # Will be updated based on conditions
    'C': 68.15,
    'D': 68.15,
    'E': 70.15,  # Will be updated based on conditions
}

# Initialize the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, 6), domain=pyo.NonNegativeIntegers)

# Obje

### Code Executability

In [12]:
import pyomo.environ as pyo

# Sample data
total_units = 239600480
max_supply = {
    'A': 33000000,
    'B': 160000000,
    'C': 165600000,
    'D': 12000000,
    'E': 77000000,
}
setup_cost = {
    'A': 0,
    'B': 0,  # Will be updated based on conditions
    'C': 0,
    'D': 0,
    'E': 0,  # Will be updated based on conditions
}
unit_cost = {
    'A': 68.15,
    'B': 0,  # Will be updated based on conditions
    'C': 68.15,
    'D': 68.15,
    'E': 70.15,  # Will be updated based on conditions
}

# Initialize the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, 6), domain=pyo.NonNegativeIntegers)

# Objective function
def obj_rule(model):
    return (
        setup_cost['A'] + unit_cost['A'] * model.x[1] / 1000 +
        setup_cost['B'] + unit_cost['B'] * model.x[2] / 1000 +
        setup_cost['C'] + unit_cost['C'] * model.x[3] / 1000 +
        setup_cost['D'] + unit_cost['D'] * model.x[4] / 1000 +
        setup_cost['E'] + unit_cost['E'] * model.x[5] / 1000
    )
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def supply_constraint_rule(model, i):
    return model.x[i] <= max_supply[list(max_supply.keys())[i - 1]]
model.supply_constraints = pyo.Constraint(range(1, 6), rule=supply_constraint_rule)

def total_units_constraint_rule(model):
    return sum(model.x[i] for i in range(1, 6)) == total_units
model.total_units_constraint = pyo.Constraint(rule=total_units_constraint_rule)

# Special conditions for vendor B
def vendor_B_conditions_rule(model):
    if 22000000 <= model.x[2] <= 70000000:
        setup_cost['B'] = 125804.84
        unit_cost['B'] = 68.099
    elif 70000001 <= model.x[2] <= 100000000:
        setup_cost['B'] = 269304.84
        unit_cost['B'] = 66.049
    elif 100000001 <= model.x[2] <= 150000000:
        setup_cost['B'] = 464304.84
        unit_cost['B'] = 64.099
    elif 150000001 <= model.x[2] <= 160000000:
        setup_cost['B'] = 761304.84
        unit_cost['B'] = 62.119
model.vendor_B_conditions = pyo.Constraint(rule=vendor_B_conditions_rule)

# Special conditions for vendor E
def vendor_E_conditions_rule(model):
    if 0 <= model.x[5] <= 42000000:
        setup_cost['E'] = 0
        unit_cost['E'] = 70.150
    elif 42000001 <= model.x[5] <= 77000000:
        setup_cost['E'] = 84000
        unit_cost['E'] = 68.150 - 0.0005 * ((model.x[5] - 42000001) // 1000000)
model.vendor_E_conditions = pyo.Constraint(rule=vendor_E_conditions_rule)

# Solve the optimization problem
solver = pyo.SolverFactory('glpk')
solution = solver.solve(model)

# Print the optimal solution
if solution.solver.status == pyo.SolverStatus.ok:
    print("Optimal solution found")
    print("Total cost:", pyo.value(model.obj))
    print("Units from each vendor:")
    for i in range(1, 6):
        print(f"Vendor {list(max_supply.keys())[i - 1]}: {pyo.value(model.x[i])}")
else:
    print("No optimal solution found")

ERROR:pyomo.core:Rule failed when generating expression for Constraint vendor_B_conditions with index None:
PyomoException: Cannot convert non-constant Pyomo expression (22000000  <=  x[2]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.
ERROR:pyomo.core:Constructing component 'vendor_B_conditions' from data=None failed:
    PyomoException: Cannot convert non-constant Pyomo expression (22000000  <=  x[2]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in

PyomoException: Cannot convert non-constant Pyomo expression (22000000  <=  x[2]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### Solution Correctness